## OpenAI agents Playground

# 🤖 OpenAI Agents Project

## 🎯 Long-term Goals
- 📚 Learn and understand OpenAI's agent architecture and capabilities
- 🔧 Create custom agents for specific use cases and domains
- 🔄 Integrate agents into other projects and workflows
- 🧩 Experiment with multi-agent systems and orchestration

## 🚀 Features
- Agent-based architecture with specialized roles
- Handoff system for intelligent task routing
- Guardrails for safe and controlled agent behavior
- Async execution for efficient processing

## 💡 Use Cases
- Educational assistants and tutoring
- Research and information gathering
- Workflow automation
- Content generation and curation

## 🛠️ Technologies Used
- OpenAI's agent framework
- Python async programming
- Pydantic for data validation
- Jupyter for interactive development

#### install

In [19]:
# ! pip install openai-agents nest_asyncio


### imports

In [28]:
from pydantic import BaseModel
import dotenv
import nest_asyncio
from agents import Agent, Runner, GuardrailFunctionOutput
from openai.types.responses import ResponseTextDeltaEvent
import asyncio
from typing import AsyncGenerator
from IPython.display import display, Markdown

##### Apply nest_asyncio to allow asyncio event loops to be nested

In [2]:
nest_asyncio.apply()

#### Load environment variables

In [3]:
dotenv.load_dotenv()

True

#### First Agent - Math Agent

In [4]:
math_tutor_agent = Agent(
    name="Math Tutor",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

#### Second and Third Agents - History and Science Agents

In [5]:
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

science_tutor_agent = Agent(
    name="Science Tutor",
    handoff_description="Specialist agent for science questions",
    instructions="You provide help with science problems. Explain concepts clearly and provide examples.",
)

### Handoffs definition

In [6]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, science_tutor_agent, math_tutor_agent]
)

### Agents Orchestration

In [27]:
async def ask_ai(q: str) -> AsyncGenerator[str, None]:
    result = Runner.run_streamed(triage_agent, input=q)
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            yield event.data.delta
    

### Adding Guardrails

In [8]:
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)

async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

###### question

In [9]:
question: str = "What is the Pythagorean theorem?"

#### run main()

In [34]:
if __name__ == "__main__":
    async def main():
        async for chunk in ask_ai(question):
            print(chunk, end="", flush=True)
    asyncio.run(main())

The Pythagorean theorem is a fundamental principle in geometry that relates the lengths of the sides of a right triangle. It states:

\[ a^2 + b^2 = c^2 \]

Here’s what each of the components means:

- \( a \) and \( b \) are the lengths of the two legs of the right triangle.
- \( c \) is the length of the hypotenuse, which is the side opposite the right angle and the longest side.

### How It Works:

Consider a right triangle where:
- One leg (\( a \)) is 3 units long.
- The other leg (\( b \)) is 4 units long.

To find the hypotenuse \( c \), apply the Pythagorean theorem:

\[ a^2 + b^2 = c^2 \]

\[ 3^2 + 4^2 = c^2 \]

\[ 9 + 16 = c^2 \]

\[ 25 = c^2 \]

To find \( c \), take the square root of both sides:

\[ c = \sqrt{25} \]

\[ c = 5 \]

So, in this example, the hypotenuse \( c \) is 5 units long.

### Real-World Example:

Suppose you have a ladder leaning against a wall, reaching a height of 8 feet, and the bottom of the ladder is 6 feet away from the wall. Using the Pythagorean 

#### Streaming Response (with Markdown)

In [37]:
async def stream(q: str) -> str:
    full_response: str = ""
    display_id: DisplayHandle = display(Markdown(full_response), display_id=True)
    
    async for chunk in ask_ai(q):
        full_response += chunk
        display_id.update(Markdown(full_response))
        await asyncio.sleep(0.01)
    
    return full_response

In [38]:
await stream(question)

The Pythagorean theorem is a fundamental principle in geometry that applies to right-angled triangles. It states that in such a triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides.

The theorem can be written as:

\[ a^2 + b^2 = c^2 \]

Where:
- \( c \) is the length of the hypotenuse.
- \( a \) and \( b \) are the lengths of the other two sides.

### Example:

Consider a right-angled triangle with sides \( a = 3 \), \( b = 4 \), and hypotenuse \( c \).

To find the hypotenuse, apply the Pythagorean theorem:

1. Calculate the squares of \( a \) and \( b \): 
   \[
   a^2 = 3^2 = 9
   \]
   \[
   b^2 = 4^2 = 16
   \]

2. Add these squares together:
   \[
   a^2 + b^2 = 9 + 16 = 25
   \]

3. Take the square root of the sum to find \( c \):
   \[
   c = \sqrt{25} = 5
   \]

Thus, the length of the hypotenuse \( c \) is 5.

### Application:

The Pythagorean theorem is useful in various applications such as architecture, navigation, and computer graphics, where determining distances and constructing right angles are often necessary.

'The Pythagorean theorem is a fundamental principle in geometry that applies to right-angled triangles. It states that in such a triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides.\n\nThe theorem can be written as:\n\n\\[ a^2 + b^2 = c^2 \\]\n\nWhere:\n- \\( c \\) is the length of the hypotenuse.\n- \\( a \\) and \\( b \\) are the lengths of the other two sides.\n\n### Example:\n\nConsider a right-angled triangle with sides \\( a = 3 \\), \\( b = 4 \\), and hypotenuse \\( c \\).\n\nTo find the hypotenuse, apply the Pythagorean theorem:\n\n1. Calculate the squares of \\( a \\) and \\( b \\): \n   \\[\n   a^2 = 3^2 = 9\n   \\]\n   \\[\n   b^2 = 4^2 = 16\n   \\]\n\n2. Add these squares together:\n   \\[\n   a^2 + b^2 = 9 + 16 = 25\n   \\]\n\n3. Take the square root of the sum to find \\( c \\):\n   \\[\n   c = \\sqrt{25} = 5\n   \\]\n\nThus, the length of the hypotenuse \\( c \\)